In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

In [3]:
from models.price_evaluator_xgboost_clasifier import (
    PriceClassifierXGBoostModel,
    PriceClassifierXGBoostModelHyperparams,
)
from training.hyperparameter_tuning import hyperparameter_tuning
from evaluation.evaluate_classification import evaluate_classification
from utils.classification_range_generator import generate_price_intervals
from utils.classify import classify
from utils.class_reduction import class_reduction
from data_processing.dtype_mapping import dtype_mapping

In [4]:
data = pd.read_csv("../data/processed_car_sale_ads.csv", low_memory=False)
data = dtype_mapping(data)
data.head(10)

,Index,Price,Currency,Condition,Vehicle_brand,Vehicle_model,Vehicle_generation,Production_year,Mileage_km,Power_HP,...,Shift paddles,Speed limiter,Start-Stop system,Sunroof,TV tuner,Tinted windows,Twilight sensor,USB socket,Velor upholstery,Xenon lights
0,0,86200,PLN,New,Abarth,595,Unknown,2021,1,145,...,0,0,0,0,0,0,0,0,0,0
1,1,43500,PLN,Used,Abarth,Other,Unknown,1974,59000,75,...,0,0,0,0,0,0,0,0,0,0
2,2,44900,PLN,Used,Abarth,500,Unknown,2018,52000,180,...,0,0,0,0,0,0,0,0,0,0
3,3,39900,PLN,Used,Abarth,500,Unknown,2012,29000,160,...,0,0,0,0,0,0,0,1,0,1
4,4,97900,PLN,New,Abarth,595,Unknown,2021,600,165,...,0,0,0,0,0,1,0,1,0,0
5,5,62950,PLN,Used,Abarth,595,Unknown,2016,46060,180,...,0,0,0,0,0,1,1,1,0,0
6,6,69900,PLN,Used,Abarth,595,Unknown,2021,2900,145,...,0,0,0,0,0,0,0,0,0,0
7,7,69900,PLN,Used,Abarth,500,Unknown,2019,20000,145,...,0,0,0,0,0,0,0,1,0,1
8,8,120750,PLN,New,Abarth,595,Unknown,2021,1,180,...,0,0,1,0,0,1,1,1,0,1
9,9,93100,PLN,New,Abarth,595,Unknown,2021,1,145,...,0,0,1,0,0,1,1,1,0,1


In [5]:
X = data.iloc[:, 3:]
y = data["Price"]

intervals = generate_price_intervals(y.min(), y.max(), 10000, 10000, 500)
labels = classify(y, intervals)
labels, intervals = class_reduction(labels, intervals)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

In [6]:
param_grid = {
    "learning_rate": [0.01],
    "max_depth": [7],
    "n_estimators": [200],
    "min_child_weight": [5],
    "gamma": [0.2],
    "subsample": [1.0],
    "colsample_bytree": [1.0],
    "reg_alpha": [0.01],
    "reg_lambda": [1.0],
}

In [7]:
best_model = hyperparameter_tuning(
    PriceClassifierXGBoostModel,
    PriceClassifierXGBoostModelHyperparams,
    param_grid,
    X_train,
    y_train,
    X_test,
    y_test,
)

print(best_model)
print(best_model.params)
print(best_model.eval(best_model.predict(X_test), y_test))

c:\Users\Barte\projects\wheel_of_fortune\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:36:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Barte\projects\wheel_of_fortune\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:36:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, 

Evaluated 0.00% of the params.
PriceClassifierXGBoostModel(base_score=None, booster=None, callbacks=None,
                            colsample_bylevel=None, colsample_bynode=None,
                            colsample_bytree=1.0, device='cuda',
                            early_stopping_rounds=None, enable_categorical=True,
                            eval_metric='logloss', feature_types=None,
                            gamma=0.2, grow_policy=None, importance_type=None,
                            interaction_constraints=None, learning_rate=0.01,
                            max_bin=None, max_cat_threshold=None,
                            max_cat_to_onehot=None, max_delta_step=None,
                            max_depth=7, max_leaves=None, min_child_weight=5,
                            missing=nan, monotone_constraints=None,
                            multi_strategy=None, n_estimators=200, n_jobs=None,
                            num_parallel_tree=None, objective='multi:softprob', 